# 视频动作分类的初级教程

## 写在前面

对视频数据的处理是计算机视觉领域非常重要的一部分内容。视频主要是由大量的视频帧图像所构成。相比于单一的图像，视频中多出了时间维度的信息，物体在先后帧中出现的顺序和状态等信息都非常关键。同时视频数据的数据量更大，而且相邻帧间差距较小，使得数据冗余度校高。所以在视频任务中，如何处理好时序信息、如何高效提取特征都是至关重要的问题。

本教程通过视频动作分类的例子来介绍和实现一个简单的文本分类方法。本教程通过加载和预处理数据、构造和提取特征、训练分类器得到一个可以应用的文本分类工具。

### 任务描述

视频动作分类任务是根据一段视频信息判别出视频中主要出现的动作类别。这是一个典型的分类任务。

例如当给出如下篮球视频，就希望模型能够给出打篮球这样的动作分类，或是输出该动作在数据集映射表中的编号。
![basketball](/picture/1.gif)

### 数据集



本次我们使用的数据集是精简版的UCF101数据集，可以在[这里](https://www.kesci.com/home/dataset/5f21c369d278b1002c21a8d8)下载，如果你能够科学上网也可以在[这里](https://drive.google.com/file/d/1KmbiMoDFNXc2xxfm--F_Qz2rpn4lkaZx/view?usp=sharing)。该数据在UCF101数据的基础上选择了十个动作类别的数据，在每个视频数据中提取了三帧内容并将其压缩至低分辨率。从而将整个数据规模控制到一个较小的范围，便于在各种设备上进行学习和实验。

下载后解压到```data```目录下。可以看到数据大约按照6:2:2的比例划分训练集、验证集、测试集，训练集、验证集、训练集的数据条数如下表所示。

数据集 | 训练集 | 验证集 | 测试集 | 总数
:-: | :-: | :-: | :-:  | :-:
数据量 | 7770 | 2230 | 3270 | 13270

### 运行环境

我们在Python3.7环境下测试了本教程代码。需要的第三方模块和版本包括：
```
pytorch
torchvision
PIL
numpy
scipy
```
可以使用pip或conda工具安装上述模块。推荐尝试使用GPU来加速代码的训练，这需要使用nvidia显卡并且搭建相应的cuda环境，当然只使用cpu也能完成本次的学习。

如果可以使用科学上网，我们将非常推荐使用Google的Colab平台。该平台提供了免费的服务器和GPU资源，可以直接运行本教程。

### 总体概述

本教程包括以下内容：从原始的数据文件中加载数据、对数据进行预处理、神经网络的搭建、训练分类器、结果展示。

#### 建议

建议学习时长：30分钟

建议对代码部分反复修改和尝试。

## 模型概述

视频的分类任务首先要对原数据进行信息抽取。动作信息包括空间信息和时间信息。空间信息主要就是视频中的每一帧包含的内容。而时间信息是指随着时间变化，各个动作运动的状态和位置的变化情况。

无论在图像还是视频任务中，巻积神经网络都能够表现出非常好的效果，是一种有效的特征提取工具，并且能够被广泛扩展到诸多不同的任务中去。本次教程中将主要使用巻积神经网络作为视频的信息抽取器。整个网络在CNN的基础上最终完成分类功能。

本次教程主要希望大家能够熟悉视频任务的工作流程，熟悉pytorch工具的基本用法，建立简单的CNN模型来完成对视频的分类任务。主要使用单帧图像来判别内容。

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My\ Drive/video recognition
!ls

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/My Drive/video recognition
data	  results.csv			视频动作分类的初级教程.ipynb
data.zip  视频动作分类的中级教程.ipynb


## 接下来我们就开始吧！
首先，我们导入实验所需要的第三方库：

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader,sampler,Dataset
import torchvision.datasets as dset
import torchvision.transforms as T
import timeit
from PIL import Image
import os
import numpy as np
import scipy.io
import torchvision.models.inception as inception

## 数据加载

在这一部分，我们将完成数据加载的工作。

我们将主要使用pytorch中设计好的Dataloader作为我们的数据的加载器。该加载器能够根据设定在每一次请求时自动加载一批训练数据，能够自主实现多线程加载，能够在快速加载的同时尽可能的节省内存开销。

而Dataloader类所加载的数据必须是pytorch中定义好的Dataset类，所以我们的第一步，就是将我们的数据封装成一个Dataset类。


首先，加载数据中的整体配置文件，并且打印出训练集和验证集的大小。

In [ ]:
label_mat=scipy.io.loadmat('./data/q3_2_data.mat')
label_train=label_mat['trLb']
print('train len：',len(label_train))
label_val=label_mat['valLb']
print('val len: ',len(label_val))

train len： 7770
val len:  2230


### Dataset类

Dataset类被定义在torch.utils.data.Dataset处，使用这个类的方法非常简单。我们只需要新构建一个Dataset类的子类，然后override该子类中的__len__和__getitem__方法：
```
__len__：返回当前dataset的大小。
__getitem__：返回一批数据。
```
```__len__```方法的主要作用就是给出当前数据的数量信息。而在我们最终使用时，我们首先会告诉Dataloader类要求加载一批数据，然后Dataloader类就会找到其中数据的```__getitem__```方法，该方法会返回一批数据来供我们使用。

In [ ]:
class ActionDataset(Dataset):
    """Action dataset."""

    def __init__(self,  root_dir, labels=[], transform=None):
        """
        Args:
            root_dir (string): 整个数据的路径。
            labels(list): 图片的标签。
            transform (callable, optional): 想要对数据进行的处理函数。
        """
        self.root_dir = root_dir
        self.transform = transform
        self.length=len(os.listdir(self.root_dir))
        self.labels=labels

    def __len__(self):      # 该方法只需返回数据的数量。
        return self.length*3    # 因为每个视频片段都包含3帧。

    def __getitem__(self, idx):     # 该方法需要返回一个数据。
        
        folder=idx//3+1
        imidx=idx%3+1
        folder=format(folder,'05d')
        imgname=str(imidx)+'.jpg'
        img_path = os.path.join(self.root_dir,folder,imgname)
        image = Image.open(img_path)

        if len(self.labels)!=0:
            Label=self.labels[idx//3][0]-1
        if self.transform:      # 如果要先对数据进行预处理，则经过transform函数。
            image = self.transform(image)
        if len(self.labels)!=0:
            sample={'image':image,'img_path':img_path,'Label':Label}
        else:
            sample={'image':image,'img_path':img_path}
        return sample



对子类简单的修改就完成了对数据的封装。现在我们可以通过一个循环测试一下：

In [ ]:
image_dataset=ActionDataset(root_dir='./data/trainClips/', labels=label_train,transform=T.ToTensor())
# torchvision.transforms中定义了非常多对图像的预处理方法，这里使用的ToTensor方法为将0～255的RGB值映射到0～1的Tensor类型。
for i in range(3):
    sample=image_dataset[i]
    print(sample['image'].shape)
    print(sample['Label'])
    print(sample['img_path'])
     
   

torch.Size([3, 64, 64])
0.0
./data/trainClips/00001/1.jpg
torch.Size([3, 64, 64])
0.0
./data/trainClips/00001/2.jpg
torch.Size([3, 64, 64])
0.0
./data/trainClips/00001/3.jpg


### Dateloader类
虽然在封装了Dataset类之后我们能够完成对数据的记载，但是在实际训练过程中我们还需要更多的步骤：
* 一次加载batch size大小的数据。
* 打乱数据的顺序。
* 多线程加载数据。

而这些需求已经全部被DataLoader类所实现：

In [ ]:
image_dataloader = DataLoader(image_dataset, batch_size=4,
                        shuffle=True, num_workers=4)

for i,sample in enumerate(image_dataloader):
    sample['image']=sample['image']
    print(i,sample['image'].shape,sample['img_path'],sample['Label'])
    if i>5: 
        break

0 torch.Size([4, 3, 64, 64]) ['./data/trainClips/03091/1.jpg', './data/trainClips/02197/1.jpg', './data/trainClips/05940/2.jpg', './data/trainClips/03296/3.jpg'] tensor([3., 2., 7., 3.], dtype=torch.float64)
1 torch.Size([4, 3, 64, 64]) ['./data/trainClips/02173/1.jpg', './data/trainClips/00095/2.jpg', './data/trainClips/01739/1.jpg', './data/trainClips/05789/3.jpg'] tensor([2., 0., 1., 7.], dtype=torch.float64)
2 torch.Size([4, 3, 64, 64]) ['./data/trainClips/04754/3.jpg', './data/trainClips/03760/3.jpg', './data/trainClips/03440/2.jpg', './data/trainClips/02985/1.jpg'] tensor([5., 4., 3., 3.], dtype=torch.float64)
3 torch.Size([4, 3, 64, 64]) ['./data/trainClips/02569/2.jpg', './data/trainClips/03850/3.jpg', './data/trainClips/01659/3.jpg', './data/trainClips/02833/2.jpg'] tensor([2., 4., 1., 3.], dtype=torch.float64)
4 torch.Size([4, 3, 64, 64]) ['./data/trainClips/06912/2.jpg', './data/trainClips/04246/2.jpg', './data/trainClips/00848/2.jpg', './data/trainClips/07680/1.jpg'] tensor

我们发现每一次调用都能够自动返回batch size的数据，并且数据被完全打乱。同时还实现了多线程加载，加载速度提升了很多。

所以我们使用将训练集、验证集和测试集全部封装起来：

In [ ]:
image_dataset_train=ActionDataset(root_dir='./data/trainClips/',labels=label_train,transform=T.ToTensor())

image_dataloader_train = DataLoader(image_dataset_train, batch_size=32,
                        shuffle=True, num_workers=4)
image_dataset_val=ActionDataset(root_dir='./data/valClips/',labels=label_val,transform=T.ToTensor())

image_dataloader_val = DataLoader(image_dataset_val, batch_size=32,
                        shuffle=False, num_workers=4)
image_dataset_test=ActionDataset(root_dir='./data/testClips/',labels=[],transform=T.ToTensor())

image_dataloader_test = DataLoader(image_dataset_test, batch_size=32,
                        shuffle=False, num_workers=4)

In [ ]:
dtype = torch.FloatTensor # 这是pytorch所支持的cpu数据类型中的浮点数类型。

print_every = 100   # 这个参数用于控制loss的打印频率，因为我们需要在训练过程中不断的对loss进行检测。

def reset(m):   # 这是模型参数的初始化
    if hasattr(m, 'reset_parameters'):
        m.reset_parameters()

### 数据解释和处理

在之前的步骤中我们也看到了最终要使用的数据tensor的形状，其中包含四个维度：
* N：batch size大小。
* C：每帧图像的通道数。
* H：每帧图像的高。
* W：每帧图像的宽。

这就是神经网络的输入形状。我们注意到输入过程中的每一个像素点和原图像都是一一对应的，唯一不同的就是我们将原来较大的RGB值缩放到一个小于1的浮点数，这是模型所需要的。

而在经过多个巻积层的提取之后，我们需要使用全连接层来完成最终的分类判断。此时的tensor的各个维度已经变得和原来截然不同，并且已经没有了一一对应的特点。换句话说，最终的输出的各个维度究竟代表了什么信息只有神经网络自己知道。所以tensor的形状也就不那么重要了。

所以为了便于全连接层的连接，我们定义一个Flatten函数，来将除了N以外的那些我们不知道代表什么的数据统统“展开”到一起。
 

In [ ]:
class Flatten(nn.Module):
    def forward(self, x):
        N, C, H, W = x.size() # 读取各个维度。 
        return x.view(N, -1)  # -1代表除了特殊声明过的以外的全部维度。

## 搭建第一个模型！

在这一部分我们将定义一个非常简单的CNN模型，但是其几乎覆盖了所有你需要知道的基础方法和训练步骤。

首先让我们提前来看一看这个CNN的结构：

* 7x7 巻积层，8 filters，步长为1。
* ReLU 激活层。
* 2x2 最大池化层，步长为2。
* 7x7 巻积层，16 filters，步长为1。
* ReLU 激活层。
* 2x2 最大池化层，步长为2。
* Flatten 层。
* 2x2 最大池化层，步长为2。
* 全连接层。

整体的结构是非常经典的：巻积+激活+池化结构的堆叠，逐步提取特征值。

使用pytorch定义模型结构非常的简单，只需要在Sequential函数中将所有的网络层顺序相连即可。

其中需要注意各个层之间输入输出尺寸要一致，巻积层和池化层的输出维度计算公式为N=(W-F+2P)/S+1，其中F为巻积核大小，P为图像四周的padding大小，S为步长。


In [ ]:
fixed_model_base = nn.Sequential( 
                nn.Conv2d(3, 8, kernel_size=7, stride=1), #3*64*64 -> 8*58*58
                nn.ReLU(inplace=True),
                nn.MaxPool2d(2, stride = 2),    # 8*58*58 -> 8*29*29
                nn.Conv2d(8, 16, kernel_size=7, stride=1), # 8*29*29 -> 16*23*23
                nn.ReLU(inplace=True),
                nn.MaxPool2d(2, stride = 2), # 16*23*23 -> 16*11*11
                Flatten(),
                nn.ReLU(inplace=True),
                nn.Linear(1936, 10)     # 1936 = 16*11*11
            )
# 这里模型base.type()方法是设定模型使用的数据类型，之前设定的cpu的Float类型。
# 如果想要在GPU上训练则需要设定cuda版本的Float类型。
fixed_model = fixed_model_base.type(dtype)

接下来我们做一个简单的测试，来验证我们模型的输入输出尺寸。

随即生成一个32*3*64*64大小的tensor，并将其封装为torch的Variable类型，然后输入到模型中。

通过检查输出模型，得到了32*10的tensor，其中的32代表32组数据点，10代表我们本次任务的类别数量。验证了模型的正确性。

In [ ]:
x = torch.randn(32, 3, 64, 64).type(dtype)
x_var = Variable(x.type(dtype)) # 需要将其封装为Variable类型。
ans = fixed_model(x_var)        

print(np.array(ans.size())) # 检查模型输出。
np.array_equal(np.array(ans.size()), np.array([32, 10]))   


[32 10]


True

## 训练步骤及模块

接下来就到了最重要的步骤，我们需要训练之前定义的CNN模型。

最为普遍的神经网络训练可以简单概括为以下三步：
* 将训练数据输入模型开始前向传播。
* 通过损失函数计算模型输出和标准答案之间的差距，得到loss值。
* 根据loss值反向传播，使用优化器更新模型参数。

前向传播就是完整的按照模型搭建顺序对输入进行处理，并且最终得到输出。

在得到了输出之后就可以通过损失函数计算loss，loss的数值将直接被应用到各个参数的修改过程中去，它能够直接影响模型的收敛速度和最终的收敛情况，所以损失函数在训练过程中就显得极为重要。独特的损失函数基本是每一类成熟神经网络所必须的。

优化器的选择同样是非常重要和复杂的事情，优化器直接影响每一轮训练的结果，不同的优化其会带来不同的收敛速度和收敛结果，可谓是神经网络训练的核心。其相关了非常多参数和方法，比如学习率、动量、学习率变化率等等，都是非常重要的部分。

使用pytorch训练模型将变得非常方便，pytorch基于动态图模型，具有自动求导功能，能够在前向传播过程中保留各个变量的梯度。极大的简化了手动求导的过程。

具体的训练步骤非常简单：
1. 数据输入模型得到输出。
2. 根据输出和标签计算loss。
3. optimizer.zero_grad () 优化器梯度归零。
4. loss.backward () loss反向传播。
5. optimizer.step () 优化器更新参数。

根据这个步骤，我们完成了下面的训练和验证模块：

In [ ]:
def train(model, loss_fn, optimizer, dataloader, num_epochs = 1):
    for epoch in range(num_epochs):
        print('Starting epoch %d / %d' % (epoch + 1, num_epochs))

        # 在验证集上验证模型效果
        check_accuracy(fixed_model, image_dataloader_val)
        
        model.train() # 模型的.train()方法让模型进入训练模式，参数保留梯度，dropout层等部分正常工作。
        for t, sample in enumerate(dataloader):
            x_var = Variable(sample['image'])   # 取得一个batch的图像数据。
            y_var = Variable(sample['Label'].long()) # 取得对应的标签。

            scores = model(x_var)   # 得到输出。
            
            loss = loss_fn(scores, y_var)   # 计算loss。
            if (t + 1) % print_every == 0:  # 每隔一段时间打印一次loss。
                print('t = %d, loss = %.4f' % (t + 1, loss.item()))

            # 三步更新参数。
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

def check_accuracy(model, loader):

    num_correct = 0
    num_samples = 0

    model.eval() # 模型的.eval()方法切换进入评测模式，对应的dropout等部分将停止工作。
    for t, sample in enumerate(loader):
        x_var = Variable(sample['image'])
        y_var = sample['Label']
       
        scores = model(x_var)
        _, preds = scores.data.max(1) # 找到可能最高的标签作为输出。

        num_correct += (preds.numpy() == y_var.numpy()).sum()
        num_samples += preds.size(0)
    acc = float(num_correct) / num_samples
    print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))


接下来设定本次实验选用的损失函数和优化器。

非常鼓励大家自己尝试不同损失函数和优化器带来的不同的效果。

In [ ]:
optimizer = torch.optim.RMSprop(fixed_model_base.parameters(), lr = 0.0001)

loss_fn = nn.CrossEntropyLoss()

## 训练并验证

完成了上面所有的准备工作之后，让我们正式开始模型的训练。

这里强烈建议大家自行调整各个超参数的值并反复训练，体会不同参数带来的效果和变化。

同时非常鼓励有GPU环境的读者能够将模型转移到GPU上进行训练。更改的步骤并不会很复杂，这是一个非常有帮助的尝试。


In [ ]:
torch.random.manual_seed(54321)
fixed_model.cpu()
fixed_model.apply(reset) 
fixed_model.train() 
train(fixed_model, loss_fn, optimizer,image_dataloader_train, num_epochs=5) 

不知道你是否对模型最终的结果满意，我们在模型定义之初就说个这是一个很简单的结构，它很便于理解，但绝不是效果最好的结构。

我们非常希望你在得到第一次评估结果之后能够回到模型定义和训练策略定义的位置，亲自动手改变一些结构或者数值然后观察其带来的改变。这对你理解CNN结构和训练过程会有很大帮助。

## 总结
本次教程的任务本身较为简单，但是基本覆盖了深度学习解决问题的各个过程。

经过本次教程的学习，我们希望你能够掌握pytorch的基本用法、CNN模型的搭建和训练、视频动作分类任务的简单方法等。

更多的，如果你认真的对网络参数和网络结构进行了多次调整和测试，我们希望你能够继续探索更深的问题：
* 巻积核大小的影响。
* 层数深度在速度和效果上的影响。
* 不同优化器的选择和参数设置。
* 不同激活函数的效果。
* 不同损失函数的效果。
* 数据增强方法。
* 层数越深参数越多越好吗。
* 为什么要划分出验证集和测试集，在训练过程中模型在训练集和验证集上不同的表现代表了什么。

非常希望你能够认真的思考和探索这些问题，亲自动手尝试将是学习过程中最最重要的事情。